In [ ]:
from pathlib import Path
from subprocess import check_call

import numpy as np
from numpy.testing import assert_allclose
from pymatgen.core import Lattice, Structure
from pymatgen.io.aims.inputs import AimsControlIn, AimsCube, AimsGeometryIn
from pymatgen.io.aims.outputs import AimsOutput

In [ ]:
# AIMS_CMD should be modified to match your system
AIMS_CMD = "aims.x"
AIMS_OUTPUT = "aims.out"
AIMS_SD = "species_dir"

In [ ]:
# Create test structure
structure = Structure(
    lattice=Lattice(
        np.array([[0, 2.715, 2.715], [2.715, 0, 2.715], [2.715, 2.715, 0]])
    ),
    species=["Si", "Si"],
    coords=np.array([np.zeros(3), np.ones(3) * 0.25]),
)

In [ ]:
# Create the geometry file from the structure
geo_in = AimsGeometryIn.from_structure(structure)

# Create the control.in file
cont_in = AimsControlIn(
    {
        "xc": "pw-lda",
        "relax_geometry": "trm 0.01",
        "relax_unit_cell": "full",
        "species_dir": AIMS_SD,
    }
)

# Add new parameters as if AimsControl
cont_in["k_grid"] = [1, 1, 1]

# Output options to control in automatically append the list
cont_in["output"] = "hirshfeld"
cont_in["output"] = ["eigenvectors"]

# Cube file output controlled by the AimsCube class
cont_in["cubes"] = [
    AimsCube("total_density", origin=[0, 0, 0], points=[11, 11, 11]),
    AimsCube("eigenstate_density 1", origin=[0, 0, 0], points=[11, 11, 11]),
]

In [ ]:
# Write the input files
work_dir = Path.cwd() / "workdir/"
work_dir.mkdir(exist_ok=True)

geo_in.write_file(work_dir, overwrite=True)
cont_in.write_file(structure, work_dir, overwrite=True)

In [ ]:
# Run the calculation
# with open(f"{work_dir}/{AIMS_OUTPUT}", "w") as outfile:
#     aims_run = check_call([AIMS_CMD], cwd=work_dir, stdout=outfile)

In [ ]:
# Read the aims output file and the final relaxed geometry
outputs = AimsOutput.from_outfile(f"{work_dir}/aims.out.gz")
relaxed_structure = AimsGeometryIn.from_file(f"{work_dir}/geometry.in")

# Check the results
print(outputs.get_results_for_image(-1))

Full Formula (Si8)
Reduced Formula: Si
abc   :   5.630352   5.630352   5.630352
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP            a      b      c  force
---  ----  ---------  -----  -----  -------------------------------------------------
  0  Si    -0.498224   0.5    0.5   [-1.10236868e-01  5.39644191e-08  4.70492367e-07]
  1  Si     0.5        0     -0     [-1.83812467e-02  3.53891730e-08  4.63693170e-07]
  2  Si     0          0.5   -0     [4.67871056e-03 3.57565863e-08 4.65599333e-07]
  3  Si     0          0      0.5   [4.67870957e-03 4.34255894e-08 4.38675279e-07]
  4  Si    -0.25      -0.25  -0.25  [0.02997323 0.01705022 0.01704979]
  5  Si    -0.25       0.25   0.25  [ 0.02997324 -0.0170503  -0.01705073]
  6  Si     0.25      -0.25   0.25  [ 0.02965711 -0.0165611   0.0165606 ]
  7  Si     0.25       0.25  -0.25  [ 0.02965711  0.01656101 -0.0165615 ]
